In [1]:
using Distributed
@everywhere begin 
    using ParallelTemperingMonteCarlo
    using DelimitedFiles
    using BenchmarkTools
end

In [13]:
using Random,Plots

#set random seed - for reproducibility
Random.seed!(1234)

# number of atoms
n_atoms = 13

# temperature grid
ti = 5.
tf = 16.
n_traj = 32

temp = TempGrid{n_traj}(ti,tf) 

# MC simulation details
mc_cycles = 100000 #default 20% equilibration cycles on top
mc_sample = 1  #sample every mc_sample MC cycles

#move_atom=AtomMove(n_atoms) #move strategy (here only atom moves, n_atoms per MC cycle)
displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust, eq_percentage = 1)

#moves - allowed at present: atom, volume and rotation moves (volume,rotation not yet implemented)
move_strat = MoveStrategy(atom_moves = n_atoms)  

#ensemble
ensemble = NVT(n_atoms)

#ELJpotential for neon
#c1=[-10.5097942564988, 0., 989.725135614556, 0., -101383.865938807, 0., 3918846.12841668, 0., -56234083.4334278, 0., 288738837.441765]
#elj_ne1 = ELJPotential{11}(c1)

c=[-10.5097942564988, 989.725135614556, -101383.865938807, 3918846.12841668, -56234083.4334278, 288738837.441765]
pot = ELJPotentialEven{6}(c)

#starting configurations
#icosahedral ground state of Ne13 (from Cambridge cluster database) in Angstrom
pos_ne13 = [[2.825384495892464, 0.928562467914040, 0.505520149314310],
[2.023342172678102,	-2.136126268595355, 0.666071287554958],
[2.033761811732818,	-0.643989413759464, -2.133000349161121],
[0.979777205108572,	2.312002562803556, -1.671909307631893],
[0.962914279874254,	-0.102326586625353, 2.857083360096907],
[0.317957619634043,	2.646768968413408, 1.412132053672896],
[-2.825388342924982, -0.928563755928189, -0.505520471387560],
[-0.317955944853142, -2.646769840660271, -1.412131825293682],
[-0.979776174195320, -2.312003751825495, 1.671909138648006],
[-0.962916072888105, 0.102326392265998,	-2.857083272537599],
[-2.023340541398004, 2.136128558801072,	-0.666071089291685],
[-2.033762834001679, 0.643989905095452, 2.132999911364582],
[0.000002325340981,	0.000000762100600, 0.000000414930733]]

#convert to Bohr
AtoBohr = 1.8897259886
pos_ne13 = pos_ne13 * AtoBohr

length(pos_ne13) == n_atoms || error("number of atoms and positions not the same - check starting config")

#boundary conditions 
bc_ne13 = SphericalBC(radius=12.0*AtoBohr)   #5.32 Angstrom

#starting configuration
start_config = Config(pos_ne13, bc_ne13)

#histogram information
n_bin = 100
#en_min = -0.006    #might want to update after equilibration run if generated on the fly
#en_max = -0.001    #otherwise will be determined after run as min/max of sampled energies (ham vector)

#construct array of MCState (for each temperature)
mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot; max_displ=[max_displ_atom[i],0.01,1.]) for i in 1:n_traj]

#results = Output(n_bin, max_displ_vec)
results = Output{Float64}(n_bin; en_min = mc_states[1].en_tot)



Output{Float64}(100, 0.0, 0.0, Float64[], Float64[], Float64[], Vector{Float64}[], Float64[], Float64[], Float64[], Float64[], Float64[])

In [14]:
ptmc_run!(mc_states, move_strat, mc_params, pot, ensemble, results; save_ham = false,save=true)
#plot(temp.t_grid,results.heat_cap)

# data = [results.en_histogram[i] for i in 1:n_traj]
# plot(data)

Total number of moves per MC cycle: 13



equilibration done


MC loop done.
[0.0003274332206663901, 0.00034097451808384846, 0.00035244948549771713, 0.00038565890500638536, 0.0003980335513622917, 0.00041254941004158604, 0.0004387579945671251, 0.00046105266182069345, 0.00048549186855110376, 0.0005021570901080064, 0.0005451170846650248, 0.0005775643450812362, 0.0006458749223787228, 0.0006770807764240569, 0.00071711013573714, 0.0008368237946829951, 0.0009831458227094459, 0.0011601596513707078, 0.0013070362090407607, 0.0017146656421000697, 0.0019621319526630914, 0.0030365686353476936, 0.0036432380891126223, 0.003581044352305293, 0.003610675334464941, 0.0032390163705791784, 0.0028215089701653243, 0.0028292964927889848, 0.003584854417093773, 0.003757680935178654, 0.004212266642843023, 0.004559761171251476]
done


Testing better mc_cycles

In [15]:
@everywhere function parallelmc_cycle!(mc_states, move_strat, mc_params, pot, ensemble, n_steps, a, v, r)
    #perform one MC cycle
    Threads.@threads for i_traj = 1:mc_params.n_traj
        for i_step = 1:n_steps
            #mc_states[i_traj] = mc_step!(type_moves[ran][2], type_moves[ran][1], mc_states[i_traj], ran, pot, ensemble)
            @inbounds mc_states[i_traj] = mc_step!(mc_states[i_traj], pot, ensemble, a, v, r)
        end
        #push!(mc_states[i_traj].ham, mc_states[i_traj].en_tot) #to build up ham vector of sampled energies
    end
    #parallel tempering
    if rand() < 0.1 #attempt to exchange trajectories
        n_exc = rand(1:mc_params.n_traj-1)
        mc_states[n_exc].count_exc[1] += 1
        mc_states[n_exc+1].count_exc[1] += 1
        exc_acc = exc_acceptance(mc_states[n_exc].beta, mc_states[n_exc+1].beta, mc_states[n_exc].en_tot,  mc_states[n_exc+1].en_tot)
        if exc_acc > rand()
            mc_states[n_exc].count_exc[2] += 1
            mc_states[n_exc+1].count_exc[2] += 1
            mc_states[n_exc], mc_states[n_exc+1] = exc_trajectories!(mc_states[n_exc], mc_states[n_exc+1])
        end
    end
    return mc_states
end

@everywhere function paralleltest(var)
    if var == -1
        for i in 1:100
            mc_cycle!(mc_states,move_strat,mc_params,pot,ensemble,1,1,0,0)
        end
    elseif var ==0
        Threads.@threads for i in 1:100
            mc_cycle!(mc_states,move_strat,mc_params,pot,ensemble,1,1,0,0)
        end
    elseif var == 1
        for i in 1:100
            parallelmc_cycle!(mc_states,move_strat,mc_params,pot,ensemble,1,1,0,0)
        end
    end
end


In [17]:
mc_params,mc_states,N_ignore = restart_ptmc(pot)
Random.seed!(1234)
time1 = @benchmark paralleltest(-1)
mc_params,mc_states,N_ignore = restart_ptmc(pot)
Random.seed!(1234)
time2 = @benchmark paralleltest(0)
mc_params,mc_states,N_ignore = restart_ptmc(pot)
Random.seed!(1234)
time3 = @benchmark paralleltest(1)

BenchmarkTools.Trial: 1920 samples with 1 evaluation.
 Range (min … max):  876.450 μs … 43.225 ms  ┊ GC (min … max):  0.00% … 96.81%
 Time  (median):       1.602 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):     2.593 ms ±  3.994 ms  ┊ GC (mean ± σ):  15.58% ± 10.19%

  ▇█                                                            
  ██▆▅▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▂▁▁▂ ▂
  876 μs          Histogram: frequency by time         32.7 ms <

 Memory estimate: 1.70 MiB, allocs estimate: 17775.

In [18]:
println("no parallel = ",time1," parallelthreads = ",time2," parallelcycle = ",time3)

no parallel = Trial

(1.354 ms) parallelthreads = Trial(392.705 μs) parallelcycle = Trial(876.450 μs)


In [22]:
@everywhere function paralleltest(var)
    if var == -1
        for i in 1:1000
            mc_cycle!(mc_states,move_strat,mc_params,pot,ensemble,1,1,0,0)
        end
    elseif var ==0
        Threads.@threads for i in 1:1000
            mc_cycle!(mc_states,move_strat,mc_params,pot,ensemble,1,1,0,0)
        end
    elseif var == 1
        for i in 1:1000
            parallelmc_cycle!(mc_states,move_strat,mc_params,pot,ensemble,1,1,0,0)
        end
    end
end
mc_params,mc_states,N_ignore = restart_ptmc(pot)
Random.seed!(1234)
time1 = @benchmark paralleltest(-1)
mc_params,mc_states,N_ignore = restart_ptmc(pot)
Random.seed!(1234)
time2 = @benchmark paralleltest(0)
mc_params,mc_states,N_ignore = restart_ptmc(pot)
Random.seed!(1234)
time3 = @benchmark paralleltest(1)

println("no parallel = ",time1," parallelthreads = ",time2," parallelcycle = ",time3)

no parallel = Trial(16.274 ms) parallelthreads = Trial(4.456 ms) parallelcycle = Trial(11.424 ms)


In [23]:
@everywhere curry_step(f,pot,ensemble,a,v,r) = x -> f(x,pot,ensemble,a,v,r)

@everywhere function parallelcycle!(mc_states, move_strat, mc_params, pot, ensemble, n_steps, a, v, r)
    pmap(curry_step(mc_step!,pot,ensemble,a,v,r ), mc_states)
    if rand() < 0.1 #attempt to exchange trajectories
        n_exc = rand(1:mc_params.n_traj-1)
        mc_states[n_exc].count_exc[1] += 1
        mc_states[n_exc+1].count_exc[1] += 1
        exc_acc = exc_acceptance(mc_states[n_exc].beta, mc_states[n_exc+1].beta, mc_states[n_exc].en_tot,  mc_states[n_exc+1].en_tot)
        if exc_acc > rand()
            mc_states[n_exc].count_exc[2] += 1
            mc_states[n_exc+1].count_exc[2] += 1
            mc_states[n_exc], mc_states[n_exc+1] = exc_trajectories!(mc_states[n_exc], mc_states[n_exc+1])
        end
    end
    return mc_states
end

In [24]:
mc_params,mc_states,N_ignore = restart_ptmc(pot)
Random.seed!(1234)
@benchmark begin 
    for i in 1:1000
        parallelcycle!(mc_states,move_strat,mc_params,pot,ensemble,1,1,0,0)
    end
end


BenchmarkTools.Trial: 38 samples with 1 evaluation.
 Range (min … max):  118.282 ms … 174.666 ms  ┊ GC (min … max): 0.00% … 7.30%
 Time  (median):     131.653 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   134.916 ms ±  12.574 ms  ┊ GC (mean ± σ):  2.70% ± 3.16%

    ▃ ▃▃ ▃  █  ▃█ ▃    ▃   ▃  ▃    ▃                             
  ▇▁█▁██▇█▁▇█▁▇██▇█▇▇▁▁█▁▇▁█▁▁█▁▁▁▁█▇▁▁▁▁▇▁▁▁▁▁▁▁▁▁▁▁▁▇▁▁▁▁▁▁▁▇ ▁
  118 ms           Histogram: frequency by time          175 ms <

 Memory estimate: 23.01 MiB, allocs estimate: 324162.